## 1) Executive Summary

NYC is one of the most famous tourist cities in the world. It draws 65 million tourists in 2018 according to the city’s tourism-marketing agency. During their traveling, everyone wants to be safe and joyful. For most visitors, Subway is the most frequent transportation when they travel in NYC. To help tourists travel safely, we analyze the crime activities near each subway stations, and come up with top 10 safest and most dangerous subway stations. Around top 10 safest subway stations, we pick most-updated highly-rated Airbnb listings, to help visitors to NYC choose the cozy place to stay.

To tackle our tasks, we utilize three data tables to create our program:
    - NYC Subway Entrances / Stations on NYC map
    - NYPD Complaint Locations on NYC map
    - Airbnb locations – filtered for a favorable rating location

We reduce data in light of the limited capabilities of software to process large dataset. We stratify the data so that we can study 379 Manhattan Subway stations, while we filter the Airbnb with rating above 85 in Manhattan and get 20K listings. Also, we limit the NYC Complaints for only the 2018 data, which had about 500K rows.

The first final output plots the top 10 subway stations with highest and lowest criminal on a NYC Manhattan map. The second final output shows the top 10 safest subway stations and all highly rated airbnbs within 200 meters of each of them.

From the result, we know that the most dangerous locations are not necessarily centralized, but they are spread out across Manhattan. Further we have found safest locations are spread out North to the South on the West side near Central Park of Manhattan and most dangerous locations are spread out from Midtown to Downtown stations.

To all enthusiastic travelers, we highly recommend staying near Central Park area. Where they will have lot of fun activities to do and have safe travel!

**As per our analysis these are the top 10 safest and most dangerous stations in Manhattan**

**Below is the top 10 safest subway stations and recommended Airbnb in Manhattan**

## 2) Background

>It’s always exciting when you have to travel to a big city that you have never been before, whether it’s for business or for pleasure.  Then to plan out the trip, we get in front of our laptops in hopes of booking a flight and a hotel.  However, we end up spending way too much time or eventually giving up booking the trip due to the uncertainties.  There is always this fear that, wherever we end up booking, is full of misfits like the drug dealers, muggers, etc.  

>This is the backdrop of our project.  We want to help potential tourists traveling to New York City to be less fearful and assure them that when you choose the New York locations from our list, they shouldn’t be handicapped by fear.  We will provide NYC travelers with access to our program that will pinpoint the safest locations to stay while he or she is visiting NYC, and also be able to access the safest subway stations and entrances.

>To tackle our goal, we utilize NYC Subway Entrances / Stations on NYC map and NYPD Complaint Locations on NYC map from NYC open data, and Airbnb listings from Inside Airbnb website.

## 3) Project Description

To tackle our goal, the steps involved are:
1. Acquiring the data:  
We get open data from NYC open data and Inside Airbnb website. 
2. Transforming the data:  
We use various tools, such as Jupyter Notebook, Google Colab, and SQL Workbench, and regular expression, dataframe in python pandas, to best scrub, clean, and store our data. The common fields in three different datasets are latitude and longitude, we extract the clear information out for later analysis.
3. Analyzing the data:  
We calculate the distance between each subway stations and crime complaints, to decide the number of crimes close to each subway stations. Then we generate top 10 safest and top 10 most dangerous subway stations. Moreover, we use the similar method to generate best Airbnb listings around top 10 safest subway stations.
4. Visualizing the data:  
We visualize, put our output together on the NYC map via Python Folium library to give a more intelligible result.


# Transforming the data

## The following code was used to clean up the New York Subway Station database. The steps that needed to be taken were:
- First, create a list to obtain a dataset with just the columns OBJECTID, NAME, the_geom, and Line. These columns refer to the following information: 
  - OBJECTID: ID number for each subway station and entrances 
  - NAME: The name of each subway station and entrance 
  - the_geom: the latitude and longitude of each subway station 
  - Line: the lines available at each subway station
- Next, we used regular expressions to remove unwanted text from the location data and then split the_geom column into seperate columns of latitude and longitude. We rounded the latitudes and longitudes of each complaint so that they matched the airbnb dataset location data. The numbers were rounded to 5 places after the decimal for consistency. This will be useful when we begin calculating the distance between our datasets (complaint, subway, and airbnb locations) later on. 
- The data included several of the same subway stations' different entrances, so we processed the NAME column so that we could group the subway stations with the same name, regardless of the entrance.
- We then took the latitudes and longitudes of each subway station (at all entrances) and averaged them to determine a station longitude and latitude. This was done to make the analysis easier and have a smaller dataset.


In [2]:
import pandas as pd
import numpy as np

In [4]:
from sqlalchemy import create_engine

import MySQLdb as mdb
import re
# The connect() method has four parameters. 
# The first parameter is the host, where the MySQL database is located. 
# In our case it is a localhost, e.g. our computer. 
# The second parameter is the database user name. 
# It is followed by the user's account password. 
# The final parameter is the database name.
host = 'bigdata.stern.nyu.edu'
username = 'dealingF19GB1'
password = 'dealingF19GB1!!'
database = 'dealingF19GB1'

con = mdb.connect(host, username, password, database, 
                charset='utf8', use_unicode=True);
cur = con.cursor()
cur.execute("select * from subway_entrances")
listlatlon = list()



for i in range(cur.rowcount):

    row = cur.fetchone()
    listlat = list()
    
    listlat.append(row[0])
    listlat.append(row[2])
    listlat.append(row[3])
    listlat.append(row[4])
    listlatlon.append(listlat)
print(listlatlon[0:10])


[[1, '15th St & Kings Hwy at SE corner', 'POINT (-73.95776900005806 40.6088350009905)', 'B-Q'], [2, '15th St & Avenue U at SE corner', 'POINT (-73.95582799955477 40.59899100093364)', 'B-Q'], [3, '15th St & Gravesend Neck Rd at NE corner', 'POINT (-73.9551200002249 40.595020000759654)', 'B-Q'], [4, '15th St & Sheepshead Bay Rd at SE corner', 'POINT (-73.954223999936 40.58730200083184)', 'B-Q'], [5, None, 'POINT (-73.96025800029547 40.57798600098538)', 'B-Q'], [6, 'Branch 7 St & Brighton Beach Rd at NE corner', 'POINT (-73.96052400057089 40.577931001322625)', 'B-Q'], [7, 'Branch 7 St & Brighton Beach Rd at SE corner', 'POINT (-73.96023100018317 40.577746001029006)', 'B-Q'], [8, 'Branch 7 St & Brighton Beach Rd at SE corner', 'POINT (-73.96043800015305 40.577697001130275)', 'B-Q'], [9, 'Ocean Pkwy & Brighton Beach Rd at NE corner', 'POINT (-73.96789400057325 40.57642300107101)', 'Q'], [10, 'Ocean Pkwy & Brighton Beach Rd at NE corner', 'POINT (-73.96759099960158 40.576445000451336)', 'Q']

In [7]:
entrance_needed =  pd.DataFrame(listlatlon,columns=['OBJECTID' , 'NAME', 'the_geom', 'LINE'])
entrance_needed.head(10)

,OBJECTID,NAME,the_geom,LINE
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q
4,5,None,POINT (-73.96025800029547 40.57798600098538),B-Q
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q


In [6]:
entrance_needed['LatandLong'] = entrance_needed['the_geom'].str.extract(r"\((.*?)\)", expand=False)
entrance_needed.head(10)

,OBJECTID,NAME,the_geom,LINE,LatandLong
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95776900005806 40.6088350009905
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95582799955477 40.59899100093364
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.9551200002249 40.595020000759654
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.954223999936 40.58730200083184
4,5,None,POINT (-73.96025800029547 40.57798600098538),B-Q,-73.96025800029547 40.57798600098538
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052400057089 40.577931001322625
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q,-73.96023100018317 40.577746001029006
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q,-73.96043800015305 40.577697001130275
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q,-73.96789400057325 40.57642300107101
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q,-73.96759099960158 40.576445000451336


In [24]:
entrance_needed['LatandLong'] = entrance_needed['the_geom'].str.extract(r"\((.*?)\)", expand=False)
entrance_needed.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,LatandLong
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,-73.95776900005806 40.6088350009905
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,-73.95582799955477 40.59899100093364
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,-73.9551200002249 40.595020000759654
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,-73.954223999936 40.58730200083184
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793,-73.96052400057089 40.577931001322625
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q,-73.96023,40.57775,-73.96023100018317 40.577746001029006
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q,-73.96044,40.57770,-73.96043800015305 40.577697001130275
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q,-73.96789,40.57642,-73.96789400057325 40.57642300107101
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q,-73.96759,40.57645,-73.96759099960158 40.576445000451336
10,11,Ocean Pkwy & Brighton Beach Rd at SE corner,POINT (-73.96787500037775 40.576209000557355),Q,-73.96788,40.57621,-73.96787500037775 40.576209000557355


In [25]:
entrance_needed[['Longitude','Latitude']] = entrance_needed.LatandLong.str.split(" ",expand=True,)
entrance_needed.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,LatandLong
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95776900005806,40.6088350009905,-73.95776900005806 40.6088350009905
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95582799955477,40.59899100093364,-73.95582799955477 40.59899100093364
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.9551200002249,40.595020000759654,-73.9551200002249 40.595020000759654
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.954223999936,40.58730200083184,-73.954223999936 40.58730200083184
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052400057089,40.577931001322625,-73.96052400057089 40.577931001322625
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q,-73.96023100018317,40.577746001029006,-73.96023100018317 40.577746001029006
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q,-73.96043800015305,40.577697001130275,-73.96043800015305 40.577697001130275
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q,-73.96789400057325,40.57642300107101,-73.96789400057325 40.57642300107101
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q,-73.96759099960158,40.576445000451336,-73.96759099960158 40.576445000451336
10,11,Ocean Pkwy & Brighton Beach Rd at SE corner,POINT (-73.96787500037775 40.576209000557355),Q,-73.96787500037775,40.576209000557355,-73.96787500037775 40.576209000557355


In [26]:
entrance_needed['Latitude'] = entrance_needed['Latitude'].astype(float).round(5)
entrance_needed['Longitude'] = entrance_needed['Longitude'].astype(float).round(5)
entrance_needed = entrance_needed.drop(columns=['LatandLong'])
entrance_needed

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793
...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292


In [27]:
entrance_needed = entrance_needed.dropna(axis=0, subset=['NAME'])
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793
...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292


In [28]:
entrance_needed['station'] = entrance_needed['NAME'].str.extract(r'(.*)at')
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793,Branch 7 St & Brighton Beach Rd
...,...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,NaN
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St


In [29]:
entrance_needed['entrance'] = entrance_needed['NAME'].str.extract(r'at(.*)')
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy,SE corner
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U,SE corner
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd,NE corner
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd,SE corner
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793,Branch 7 St & Brighton Beach Rd,NE corner
...,...,...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,NaN,NaN
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St,SW corner
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St,NW corner
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St,NE corner


In [30]:
index_wo_at = entrance_needed.index[(entrance_needed['NAME'].str.find(r'at')==-1)].tolist()
entrance_needed.loc[index_wo_at, 'station'] = entrance_needed.loc[index_wo_at, 'NAME']
entrance_needed[(entrance_needed['NAME'].str.find(r'at')==-1)]

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance
79,80,Norwood,POINT (-73.8789840000952 40.68164300107813),J-Z,-73.87898,40.68164,Norwood,NaN
81,82,Cleveland,POINT (-73.88571600056578 40.6795300007475),J,-73.88572,40.67953,Cleveland,NaN
83,84,Van Siclen,POINT (-73.89156599958824 40.6778640010583),J-Z,-73.89157,40.67786,Van Siclen,NaN
778,779,Church St,POINT (-74.0101649998177 40.71179300109741),A-C-E-2-3,-74.01016,40.71179,Church St,NaN
779,780,Church St,POINT (-74.01020399993797 40.71171000117648),A-C-E-2-3,-74.01020,40.71171,Church St,NaN
891,892,Pitkin,POINT (-73.87187499969077 40.67555400055863),A-C,-73.87187,40.67555,Pitkin,NaN
1081,1082,Hudson Blvd East between 33rd St & 34th St (no...,POINT (-74.00092352989824 40.75497493443128),7,-74.00092,40.75497,Hudson Blvd East between 33rd St & 34th St (no...,NaN
1082,1083,Hudson Blvd East between 33rd St & 34th St (so...,POINT (-74.00098158502907 40.75489825502214),7,-74.00098,40.75490,Hudson Blvd East between 33rd St & 34th St (so...,NaN
1486,1487,7 BENTLEY STREET,POINT (-74.25283251705811 40.51211370971486),SIR,-74.25283,40.51211,7 BENTLEY STREET,NaN
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,2nd Ave & bet 83rd & 84th St,NaN


### We noticed that the dataset contained inconsistent constructions of the names of the subway stations and entrances, so we manually manipulated the data to provide uniformity and establish unique subway station names. 

In [31]:
entrance_needed.loc[index_wo_at, 'entrance'] = 'itself'
entrance_needed.loc[entrance_needed['NAME'] == 'Hudson Blvd East between 33rd St & 34th St (north entrance)', 'station'] = 'Hudson Blvd East between 33rd St & 34th St'
entrance_needed.loc[entrance_needed['NAME'] == 'Hudson Blvd East between 33rd St & 34th St (south entrance)', 'station'] = 'Hudson Blvd East between 33rd St & 34th St'
entrance_needed.loc[entrance_needed['NAME'] == 'Hudson Blvd East between 33rd St & 34th St (north entrance)', 'entrance'] = 'north'
entrance_needed.loc[entrance_needed['NAME'] == 'Hudson Blvd East between 33rd St & 34th St (south entrance)', 'entrance'] = 'south'
entrance_needed[(entrance_needed['NAME'].str.find(r'at')==-1)]

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance
79,80,Norwood,POINT (-73.8789840000952 40.68164300107813),J-Z,-73.87898,40.68164,Norwood,itself
81,82,Cleveland,POINT (-73.88571600056578 40.6795300007475),J,-73.88572,40.67953,Cleveland,itself
83,84,Van Siclen,POINT (-73.89156599958824 40.6778640010583),J-Z,-73.89157,40.67786,Van Siclen,itself
778,779,Church St,POINT (-74.0101649998177 40.71179300109741),A-C-E-2-3,-74.01016,40.71179,Church St,itself
779,780,Church St,POINT (-74.01020399993797 40.71171000117648),A-C-E-2-3,-74.01020,40.71171,Church St,itself
891,892,Pitkin,POINT (-73.87187499969077 40.67555400055863),A-C,-73.87187,40.67555,Pitkin,itself
1081,1082,Hudson Blvd East between 33rd St & 34th St (no...,POINT (-74.00092352989824 40.75497493443128),7,-74.00092,40.75497,Hudson Blvd East between 33rd St & 34th St,north
1082,1083,Hudson Blvd East between 33rd St & 34th St (so...,POINT (-74.00098158502907 40.75489825502214),7,-74.00098,40.75490,Hudson Blvd East between 33rd St & 34th St,south
1486,1487,7 BENTLEY STREET,POINT (-74.25283251705811 40.51211370971486),SIR,-74.25283,40.51211,7 BENTLEY STREET,itself
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,2nd Ave & bet 83rd & 84th St,itself


### The majority of subway stations had several entrances, and as a result locations, so we created a singular location for each subway station by averaging their various latitudes and longitudes. Later in our analysis we will look to determine which of these stations have nearby complaints and/or airbnbs, so this was necessary to avoid repetitive analysis on the same station. 

In [32]:
avg_stat = entrance_needed.groupby('station').mean()
avg_stat
avg_stat.rename(columns={'Longitude':'avg_lon', 'Latitude':'avg_lat'}, inplace=True)
avg_stat[['avg_lon','avg_lat']]

,avg_lon,avg_lat
station,,
& Ellis St,-74.245480,40.515530
103rd St & 159th Ave,-73.830427,40.660373
103rd St & Roosevelt Ave,-73.862478,40.749875
104th St & Jamaica Ave,-73.843260,40.695150
104th St & Liberty Ave,-73.836790,40.682055
...,...,...
William St & John St,-74.007000,40.708630
William St & Pine St,-74.008800,40.706760
William St & Wall St,-74.009495,40.706390


In [33]:
entrance_needed = pd.merge(left=entrance_needed,right=avg_stat[['avg_lon','avg_lat']], left_on='station', right_on='station')
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy,SE corner,-73.957790,40.608950
1,282,15th St & Kings Hwy at NE corner,POINT (-73.95780799949603 40.609062000781535),B-Q,-73.95781,40.60906,15th St & Kings Hwy,NE corner,-73.957790,40.608950
2,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U,SE corner,-73.955830,40.598990
3,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd,NE corner,-73.955120,40.595020
4,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd,SE corner,-73.954220,40.587300
...,...,...,...,...,...,...,...,...,...,...
1894,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,2nd Ave & bet 83rd & 84th St,itself,-73.952810,40.776190
1895,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St,SW corner,-73.947470,40.784100
1896,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St,NW corner,-73.947620,40.783920
1897,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St,NE corner,-73.947985,40.782945


In [34]:
entrance_needed.to_csv('entrance_needed.csv')

**The remaining code shown below was done in the Jupyter Notebook**

### Thinking about the perspective of a potential traveler to NYC, we chose to focus our analysis on the Manahattan borough as it is the most accesible to the majority of the city's famous tourist attractions. (This also had the side benefit of significantly shortening our analysis run time). Once we isolated the data to complaints that occured in Manhattan we removed excess columns that we would not be using for analysis going forward - leaving us with just latitude and longitude rounded to 5 numbers after the decimal and the name of the subway stations.

In [9]:
subway_data = pd.read_csv('entrance_needed.csv')
subway_data

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
0,0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy,SE corner,-73.957790,40.608950
1,1,282,15th St & Kings Hwy at NE corner,POINT (-73.95780799949603 40.609062000781535),B-Q,-73.95781,40.60906,15th St & Kings Hwy,NE corner,-73.957790,40.608950
2,2,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U,SE corner,-73.955830,40.598990
3,3,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd,NE corner,-73.955120,40.595020
4,4,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd,SE corner,-73.954220,40.587300
...,...,...,...,...,...,...,...,...,...,...,...
1894,1894,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,2nd Ave & bet 83rd & 84th St,itself,-73.952810,40.776190
1895,1895,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St,SW corner,-73.947470,40.784100
1896,1896,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St,NW corner,-73.947620,40.783920
1897,1897,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St,NE corner,-73.947985,40.782945


In [10]:
north = 40.87700
south = 40.70000
west = -74.03000
east = -73.90000
subway_needed = subway_data[((subway_data['Latitude']>south) & (subway_data['Latitude']<north))]
subway_needed = subway_needed[((subway_data['Longitude']>west) & (subway_data['Longitude']<east))]
subway_needed['avg_lat'] = subway_needed['avg_lat'].round(5)
subway_needed['avg_lon'] = subway_needed['avg_lon'].round(5)
subway_needed = subway_needed[['station','avg_lat','avg_lon']]
subway_needed.drop_duplicates(inplace=True)
subway_needed

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,station,avg_lat,avg_lon
96,Flushing Ave & Broadway,40.70070,-73.94196
99,Moore St & Broadway,40.70434,-73.94827
101,Hooper St & Broadway,40.70718,-73.95435
103,Marcy Ave & Broadway,40.70841,-73.95792
109,Norfolk St & Delancey St,40.71848,-73.98742
...,...,...,...
1891,2nd Ave & 86th St,40.77781,-73.95148
1894,2nd Ave & bet 83rd & 84th St,40.77619,-73.95281
1895,2nd Ave & 96th St,40.78410,-73.94747
1896,2nd Ave & 95th St,40.78392,-73.94762


In [11]:
subway_needed.to_csv('subway_needed_manhattan.csv',index=False)

## The following code was used to clean up the NYC Complaint database. The steps that needed to be taken were:

- First, create a table (labeled: complaint_needed) to obtain a dataset with just the columns CMPNT_NUM, Latitude, Longitude, BORO_NM, LAW_CAT_CD, OFNS_DESC, CMPLNT_FR_DT, CMPLNT_FR_TM. These columns refer to the following information: 
  - CMPNT_NUM: The complaint number
  - Latitude: The latitude of the complaint 
  - Longitude: The longitude of the complaint
  - BORO_NM: The boroughs the complaint occured in
  - LAW_CAT_CD: The category of the specific complaint (Violation, Midsdemeanor, of Felony)
  - OFNS_DESC: A description of the offense the complaint was made about
  - CMPLNT_FR_DT: The date of the complaint 
  - CMPLNT_FR_TM: The time of the complaint
- Next, we rounded the latitudes and longitudes of each complaint so that they matched the airbnb dataset location data. The numbers were rounded to 5 places after the decimal for consistency. This will be useful when we begin calculating the distance between our datasets (complaint, subway, and airbnb locations) later on. 
- Any complaints that did not have location (latitude and longitude) data were dropped out of the dataset
- Additionally, we looked to focus on recent complaints for our analysis on safe areas for travelers, so we restricted the data to 2018.

**The cleaning of the complaint data was done in Google Colab**

In [39]:
import pandas as pd
import numpy as np

In [ ]:
complaintdata = pd.read_csv('NYC_ComplainData.csv')
# complaintdata.head()

In [ ]:
# complaintdata.columns
complaint_needed = complaintdata[['CMPLNT_NUM', 'Latitude', 'Longitude', 'BORO_NM', 'LAW_CAT_CD', 'OFNS_DESC', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM']]
complaint_needed

In [ ]:
complaint_needed['CMPLNT_FR_DT'] = pd.to_datetime(complaint_needed['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce')

In [ ]:
complaint_needed = complaint_needed.dropna(axis=0, subset=['Latitude', 'Longitude'])
complaint_needed

In [ ]:
start_date = '2018-01-01'
mask = (complaint_needed['CMPLNT_FR_DT'] > start_date)
complaint_needed = complaint_needed.loc[mask]
complaint_needed

In [ ]:
complaint_needed['Latitude'] = complaint_needed['Latitude'].round(5)
complaint_needed['Longitude'] = complaint_needed['Longitude'].round(5)
complaint_needed

In [ ]:
complaint_needed.to_csv('complaint_needed.csv')

**The remaining code shown below was done in the Jupyter Notebook**

### Thinking about the perspective of a potential traveler to NYC, we chose to focus our analysis on the Manahattan borough as it is the most accesible to the majority of the city's famous tourist attractions. (This also had the side benefit of significantly shortening our analysis run time). Once we isolated the data to complaints that occured in Manhattan we removed excess columns that we would not be using for analysis going forward - leaving us with just latitude and longitude rounded to 5 numbers after the decimal.

In [42]:
complaint_data = pd.read_csv('complaint_needed.csv')
complaint_data

,CMPLNT_NUM,Latitude,Longitude,BORO_NM,LAW_CAT_CD,OFNS_DESC,CMPLNT_FR_DT,CMPLNT_FR_TM
0,774621657,40.823836,-73.861624,BRONX,FELONY,ROBBERY,2018-10-08,12:30:00
1,232548146,40.667412,-73.986123,BROOKLYN,MISDEMEANOR,PETIT LARCENY,2018-08-24,11:00:00
2,452701517,40.760427,-73.912285,QUEENS,FELONY,DANGEROUS WEAPONS,2018-03-30,16:55:00
3,620357753,40.661923,-73.748389,QUEENS,MISDEMEANOR,PETIT LARCENY,2018-10-02,16:00:00
4,110535568,40.865428,-73.867865,NaN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,2018-08-14,14:20:00
...,...,...,...,...,...,...,...,...
450999,965046527,40.672071,-73.876950,BROOKLYN,MISDEMEANOR,CRIMINAL TRESPASS,2018-06-10,02:23:00
451000,854641518,40.758468,-73.866738,QUEENS,VIOLATION,HARRASSMENT 2,2018-11-04,06:45:00
451001,311122873,40.743141,-73.988421,MANHATTAN,MISDEMEANOR,CRIMINAL TRESPASS,2018-09-14,21:30:00
451002,315442524,40.628552,-74.076470,STATEN ISLAND,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,2018-04-20,23:00:00


In [43]:
complaint_needed = complaint_data[complaint_data['BORO_NM']=='MANHATTAN']
complaint_needed['Latitude'] = complaint_needed['Latitude'].round(5)
complaint_needed['Longitude'] = complaint_needed['Longitude'].round(5)
complaint_needed = complaint_needed[['Latitude','Longitude']]
complaint_needed

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Latitude,Longitude
7,40.75043,-73.98928
18,40.75664,-73.98837
23,40.72543,-73.99208
49,40.80277,-73.93860
50,40.76187,-73.96636
...,...,...
450983,40.75160,-73.99204
450989,40.72533,-73.97267
450993,40.74377,-73.98797
451001,40.74314,-73.98842


### When reviewing the data, we noticed that several locations (specific latitude and longitudes) had more than one complaint, so we created a count column for those cases to group them together. 

In [44]:
complaint_count = complaint_needed.groupby(['Latitude','Longitude']).size()
complaint_count = complaint_count.reset_index()
complaint_count = complaint_count.rename(columns = {0:'count'})
complaint_count

,Latitude,Longitude,count
0,40.61733,-73.99032,1
1,40.62817,-73.94136,1
2,40.67136,-73.88181,1
3,40.68370,-74.00051,1
4,40.68887,-73.94453,1
...,...,...,...
8829,40.87277,-73.91782,4
8830,40.87303,-73.91846,2
8831,40.87307,-73.91176,1
8832,40.87323,-73.91884,1


### We then merged the complaint count (number of crimes) data back to our complaint dataframe.

In [45]:
complaint_needed = pd.merge(complaint_needed, complaint_count, on=['Latitude','Longitude'])
complaint_needed.drop_duplicates(inplace=True)
complaint_needed

,Latitude,Longitude,count
0,40.75043,-73.98928,1238
1238,40.75664,-73.98837,298
1536,40.72543,-73.99208,30
1566,40.80277,-73.93860,12
1578,40.76187,-73.96636,255
...,...,...,...
111303,40.77238,-73.96079,1
111304,40.75931,-73.96220,1
111305,40.71040,-73.99518,1
111306,40.82665,-73.94306,1


In [46]:
complaint_needed.to_csv('complaint_needed_manhattan.csv',index=False)

# Analyzing the data

## Now that we have cleaned and processed the NYC Subway and Complaint databases, we wanted to determine which of the subway stations might be considered the safest or most dangerous based on the number of complaints occuring within their nearby vicinity. This will be useful for the NYC visitor as it can help inform them on where they might want to stay in NYC that is both safe and close to a transit line for getting around. To do this we took the following steps:
- First we needed to count the number of crimes (complaints) that occured within 100 meters of each subway station
- Then, we selected the 10 stations with the highest occurance of complaints nearby to be the most dangerous and the 10 stations with the lowest occurence of complaints nearby to be the safest.
- Finally, we plotted both of these top 10 lists on a map to display visually where these locations are in Manhattan. 

In [4]:
import pandas as pd
import numpy as np

In [49]:
complaint_data = pd.read_csv('complaint_needed_manhattan.csv')
complaint_data

,Latitude,Longitude,count
0,40.75043,-73.98928,1238
1,40.75664,-73.98837,298
2,40.72543,-73.99208,30
3,40.80277,-73.93860,12
4,40.76187,-73.96636,255
...,...,...,...
8829,40.77238,-73.96079,1
8830,40.75931,-73.96220,1
8831,40.71040,-73.99518,1
8832,40.82665,-73.94306,1


In [50]:
subway_data = pd.read_csv('subway_needed_manhattan.csv')
subway_data

,station,avg_lat,avg_lon
0,Flushing Ave & Broadway,40.70070,-73.94196
1,Moore St & Broadway,40.70434,-73.94827
2,Hooper St & Broadway,40.70718,-73.95435
3,Marcy Ave & Broadway,40.70841,-73.95792
4,Norfolk St & Delancey St,40.71848,-73.98742
...,...,...,...
374,2nd Ave & 86th St,40.77781,-73.95148
375,2nd Ave & bet 83rd & 84th St,40.77619,-73.95281
376,2nd Ave & 96th St,40.78410,-73.94747
377,2nd Ave & 95th St,40.78392,-73.94762


### In order to determine the distance between subway station and complaint locations, we used the Geopy package to calculate the distance between two locations by latitude and longitude.

In [51]:
from geopy.distance import geodesic
def geodistance(location1, location2):
    distance = geodesic(location1,location2).m
    return distance

###  We then loopped through the data to calculate the number of crimes/complaints that occured within close proximity to each subway station. 

**If the crimes/complaints occured within 100 meters of a station, they were counted.**

In [ ]:
import time
count = 0
start = time.time()
for id1,row1 in subway_needed.iterrows():
    number_of_complaint = 0
    location1 = (row1['avg_lat'], row1['avg_lon'])
    
    for id2,row2 in complaint_needed.iterrows():
        location2 = (row2['Latitude'], row2['Longitude'])
        if(geodistance(location1,location2) <= 100):
            number_of_complaint = number_of_complaint + row2['count']
#             print(number_of_complaint)
    
    subway_needed.loc[id1,'complaint'] = number_of_complaint
    count = count + 1
#     if count > 19:
#         break
    
subway_needed
end = time.time()
print((end-start)/60)

In [16]:
complaint_per_subway = subway_needed[subway_needed['complaint']!=0]
complaint_per_subway

,station,avg_lat,avg_lon,complaint
40,Adam C. Powell Blvd & 149th St,40.82407,-73.93698,96.0
41,Lenox Ave & 145th St,40.82044,-73.93624,73.0
45,Lenox Ave & 135th St,40.81411,-73.94087,150.0
49,Lenox Ave & 125th St,40.80778,-73.94549,251.0
53,Lenox Ave & 116th St,40.80203,-73.94969,176.0
...,...,...,...,...
1891,2nd Ave & 86th St,40.77781,-73.95148,86.0
1894,2nd Ave & bet 83rd & 84th St,40.77619,-73.95281,37.0
1895,2nd Ave & 96th St,40.78410,-73.94747,96.0
1896,2nd Ave & 95th St,40.78392,-73.94762,101.0


In [17]:
complaint_per_subway.to_csv('complaint_for_manhattan_subways.csv',index=False)

In [52]:
complaint_per_subway = pd.read_csv('complaint_for_manhattan_subways.csv')

### When initially reviewing our top 10 most dangerous and safe subway stations based on nearby crimes/complaints, we noticed there were some repetitive data (the same station appearing multiple times) in our lists. To correct for this we manually adjusted the lists. 

The reason why we had to do this manual modification was due to the data quality of the subway data we have. The way the names of stations were contructed are not consistent, so we had to plot it out to see if any of them were too close to one another. We then went to Google Maps to check around the latitudes and longitudes of each station on our top 10 lists and confirm there was only one subway station at that location. 

In [53]:
# Lexington Avenue & 125th Street(119) & Lexington Ave & 125th St(120) should be the same station
# Broadway & 43rd St(226) & 8th Ave & 40th St(140) & 8th Ave & 43rd St(230) & 7th Ave & 42nd St(51) & 8th Ave & 42nd St(139) & Broadway & 42nd St(182) & 7th Ave & 41st St(181) should be the same station
# Church St & Dey St(248) & Church St & Cortlandt St(263) should be the same station
most_dangerous = complaint_per_subway.sort_values(by='complaint', ascending=False)[:18]
# most_dangerous
most_dangerous = most_dangerous.drop([120,140,230,51,139,182,263,181])
most_dangerous

,station,avg_lat,avg_lon,complaint
119,Lexington Avenue & 125th Street,40.80441,-73.93723,736.0
68,Broadway & 14th St,40.73497,-73.99110,702.0
226,Broadway & 43rd St,40.75648,-73.98631,611.0
129,Saint Nicholas Ave & 125th St,40.81077,-73.95260,526.0
23,Broadway & Prince St,40.72424,-73.99779,490.0
14,6th Ave & 34th St,40.74974,-73.98756,455.0
191,Varick St & North Moore St,40.71990,-74.00677,454.0
177,Broadway & 95th St,40.79447,-73.97205,427.0
248,Church St & Dey St,40.71097,-74.01070,411.0
137,8th Ave & 58th St,40.76731,-73.98270,389.0


In [54]:
names_dangerous = ['125 Street Station - 4,5,6','14 Street - Union Sq Station','Times Sq - 42 St Station',
                   '125 Street Station - A,B,C,D','Prince Street Station','34 St - Herald Sq Station',
                   'Franklin Street Station','96 Street Station - 1,2,3','Fulton Street Station','59 St - Columbus Circle ']
names_dangerous

['125 Street Station - 4,5,6',
 '14 Street - Union Sq Station',
 'Times Sq - 42 St Station',
 '125 Street Station - A,B,C,D',
 'Prince Street Station',
 '34 St - Herald Sq Station',
 'Franklin Street Station',
 '96 Street Station - 1,2,3',
 'Fulton Street Station',
 '59 St - Columbus Circle ']

In [55]:
most_dangerous['subway']=names_dangerous
most_dangerous

,station,avg_lat,avg_lon,complaint,subway
119,Lexington Avenue & 125th Street,40.80441,-73.93723,736.0,"125 Street Station - 4,5,6"
68,Broadway & 14th St,40.73497,-73.99110,702.0,14 Street - Union Sq Station
226,Broadway & 43rd St,40.75648,-73.98631,611.0,Times Sq - 42 St Station
129,Saint Nicholas Ave & 125th St,40.81077,-73.95260,526.0,"125 Street Station - A,B,C,D"
23,Broadway & Prince St,40.72424,-73.99779,490.0,Prince Street Station
14,6th Ave & 34th St,40.74974,-73.98756,455.0,34 St - Herald Sq Station
191,Varick St & North Moore St,40.71990,-74.00677,454.0,Franklin Street Station
177,Broadway & 95th St,40.79447,-73.97205,427.0,"96 Street Station - 1,2,3"
248,Church St & Dey St,40.71097,-74.01070,411.0,Fulton Street Station
137,8th Ave & 58th St,40.76731,-73.98270,389.0,59 St - Columbus Circle


In [56]:
# Alexander Ave & 138th St(118) not in Manhattan
# Fort Washington Ave & 192nd St(168) too far from most tourist spots
# Central Park West & 87th St(85) & Central Park West & 88th St(84) & Central Park West & 86th St(133) & 1039 should be the same location
# Fort Washington Ave & Margaret Corbin Dr(169) too far from most touris spots
# Central Park West & 81st St(134) & Central Park West & 77th St(86) should be the same station
# Central Park West & 97th St(83) & Central Park West & 96th St(132) should be the same station
# West Broadway & Franklin St(59) & Varick St & Franklin St(192) should be the same station
# West Broadway & Franklin St(59) should be Varick St & North Moore St in the most dangerous part
# Broadway & 190th St(36) too far from most tourist spots
safest = complaint_per_subway.sort_values(by='complaint')[:20]
safest = safest.drop([118,168,133,169,84,86,132,192,59,36])
safest

,station,avg_lat,avg_lon,complaint
170,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0
193,Greenwich St & Morris St,40.70649,-74.01414,8.0
271,2nd Ave & 94th St,40.78294,-73.94798,11.0
85,Central Park West & 87th St,40.78612,-73.96901,11.0
134,Central Park West & 81st St,40.78212,-73.97188,15.0
87,Central Park West & 70th St,40.77494,-73.97714,17.0
83,Central Park West & 97th St,40.79244,-73.96437,17.0
77,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0
111,6th Ave & 16th St,40.73869,-73.99587,23.0
158,6th Ave & 49th St,40.75929,-73.98085,25.0


In [57]:
names_safest=['34 Street - Hudson Yards Station','Bowling Green Station','96 Street Station - Q',
              '86 Street Station','81 Street - Museum of Natural History Station',
              '72 Street Station','96 Street Station - B,C','135 Street Station',
              '14 Street Station','47-50 Streets Rockefeller Center Station']
names_safest

['34 Street - Hudson Yards Station',
 'Bowling Green Station',
 '96 Street Station - Q',
 '86 Street Station',
 '81 Street - Museum of Natural History Station',
 '72 Street Station',
 '96 Street Station - B,C',
 '135 Street Station',
 '14 Street Station',
 '47-50 Streets Rockefeller Center Station']

In [58]:
safest['subway'] = names_safest
safest

,station,avg_lat,avg_lon,complaint,subway
170,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0,34 Street - Hudson Yards Station
193,Greenwich St & Morris St,40.70649,-74.01414,8.0,Bowling Green Station
271,2nd Ave & 94th St,40.78294,-73.94798,11.0,96 Street Station - Q
85,Central Park West & 87th St,40.78612,-73.96901,11.0,86 Street Station
134,Central Park West & 81st St,40.78212,-73.97188,15.0,81 Street - Museum of Natural History Station
87,Central Park West & 70th St,40.77494,-73.97714,17.0,72 Street Station
83,Central Park West & 97th St,40.79244,-73.96437,17.0,"96 Street Station - B,C"
77,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0,135 Street Station
111,6th Ave & 16th St,40.73869,-73.99587,23.0,14 Street Station
158,6th Ave & 49th St,40.75929,-73.98085,25.0,47-50 Streets Rockefeller Center Station


## check if any of the subway results we have are actually the same subway due to data quality issue

In [59]:
subway = pd.read_csv('entrance_needed.csv')
subway['avg_lat'] = subway['avg_lat'].round(5)
subway['avg_lon'] = subway['avg_lon'].round(5)
subway[subway['avg_lat']==40.71990]

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
1254,1254,1183,Varick St & North Moore St at NE corner (exit ...,POINT (-74.00662000050136 40.71991800134944),1,-74.00662,40.71992,Varick St & North Moore St,NE corner (exit only),-74.00677,40.7199
1255,1255,1186,Varick St & North Moore St at NW corner (exit ...,POINT (-74.00692499992861 40.71988800093658),1,-74.00692,40.71989,Varick St & North Moore St,NW corner (exit only),-74.00677,40.7199


In [60]:
subway = pd.read_csv('entrance_needed.csv')
subway['avg_lat'] = subway['avg_lat'].round(5)
subway['avg_lon'] = subway['avg_lon'].round(5)
subway[subway['avg_lat']==40.71900]

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
337,337,277,West Broadway & Franklin St at SW corner,POINT (-74.00667500031243 40.71900100095171),1,-74.00668,40.719,West Broadway & Franklin St,SW corner,-74.00668,40.719


In [61]:
subway = pd.read_csv('entrance_needed.csv')
subway['avg_lat'] = subway['avg_lat'].round(5)
subway['avg_lon'] = subway['avg_lon'].round(5)
subway[subway['avg_lat']==40.79447]

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
1217,1217,1132,Broadway & 95th St at N corner,POINT (-73.97204199949016 40.79432900130719),1-2-3,-73.97204,40.79433,Broadway & 95th St,N corner,-73.97205,40.79447
1218,1218,1458,Broadway & 95th St at N corner,POINT (-73.97205300014681 40.79461100095195),1-2-3,-73.97205,40.79461,Broadway & 95th St,N corner,-73.97205,40.79447


In [62]:
subway = pd.read_csv('entrance_needed.csv')
subway['avg_lat'] = subway['avg_lat'].round(5)
subway['avg_lon'] = subway['avg_lon'].round(5)
subway[subway['avg_lat']==40.78294]

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
1897,1897,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St,NE corner,-73.94798,40.78294
1898,1898,1932,2nd Ave & 94th St at SW corner,POINT (-73.94823792460468 40.78296690878563),Q,-73.94824,40.78297,2nd Ave & 94th St,SW corner,-73.94798,40.78294


In [63]:
subway = pd.read_csv('entrance_needed.csv')
subway['avg_lat'] = subway['avg_lat'].round(5)
subway['avg_lon'] = subway['avg_lon'].round(5)
subway[subway['avg_lat']==40.79244]

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
439,439,373,Central Park West & 97th St at SW corner,POINT (-73.96442299966145 40.792285001035324),B-C,-73.96442,40.79229,Central Park West & 97th St,SW corner,-73.96437,40.79244
440,440,374,Central Park West & 97th St at NW corner,POINT (-73.96431599945484 40.792591001187866),B-C,-73.96432,40.79259,Central Park West & 97th St,NW corner,-73.96437,40.79244


## Output our final result of top 10 most dangerous and top 10 safest subway stations

In [64]:
most_dangerous.to_csv('most_dangerous_final.csv',index=False)
safest.to_csv('safest_final.csv',index=False)

# Visualizing the data

## Below is a visualization of the top 10 most dangerous subway stations (determined by highest number of complaints within 100m) on the map

In [65]:
import folium
fmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
# fmap
for name, row in most_dangerous.iterrows():
    size = row['complaint']/50
    color = 'red'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)
    
fmap

## Below is a visualization of the top 10 safest subway stations (determined by lowest number of complaints within 100m) on the map

In [66]:
import folium
fmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
# fmap
for name, row in safest.iterrows():
    size = 4
    color = 'blue'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)
    
fmap

## put top 10 dangerous and top 10 safest subway stations on the same map

In [67]:
import folium
fmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
# fmap
for name, row in most_dangerous.iterrows():
    size = row['complaint']/50
    color = 'red'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)

for name, row in safest.iterrows():
    size = 4
    color = 'blue'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)
    
fmap

# Transforming the data

## The following code was used to clean up the New York Airbnb database. The steps that needed to be taken were:
- First, create a list to obtain a dataset with just the columns id	name, neighbourhood_group_cleansed, neighbourhood_cleansed	neighbourhood, latitude, longitude, review_scores_rating, and number_of_reviews. These columns refer to the following information: 
  - id name: ID number for each Airbnb 
  - neighbourhood_group_cleansed: The borough location of each airbnb
  - neighbourhood_cleansed: The neighborhood location of each airbnb
  - neighbourhood: Additional neighborhood information of each airbnb
  - latitude: The latitude of each Airbnb 
  - longitude: The longitude of each Airbnb
  - review_scores_rating: The average review rating of each Airbnb 
  - number_of_reviews: The amount of reviews each Airbnb has recieved 
- Then, the data was filtered just show the best Airbnb locations based on ratings and number of reviews.
- Similarly to the complaint and subway location datasets, the data was limited to Manhattan locations. 


In [68]:
import pandas as pd
import numpy as np



### As a traveler looking to come to NYC and stay in a short term rental there are thousands of options and, as you can imagine, a large range in quality of the Airbnb listings. Thankfully, the Airbnb review process is a great resource in vetting different options when looking at a place to stay while traveling. We wanted our tool to simplify the process of sorting through tons of Airbnb options so we limited the data to Airbnb locations with average review scores greater than 85 (out of 100) and that had at least 10 reviews. This ensures that the Airbnb options presented in this study have been confirmed to be of good quality by a significant amount of previous renters. 

In [2]:
airbnbdata = pd.read_csv('filter_airbnb.csv')

airbnb_table = airbnbdata[['id','name','neighbourhood_group_cleansed','neighbourhood_cleansed', 'neighbourhood', 'latitude','longitude','review_scores_rating','number_of_reviews']]
airbnb_table


,id,name,neighbourhood_group_cleansed,neighbourhood_cleansed,neighbourhood,latitude,longitude,review_scores_rating,number_of_reviews
0,3831,Cozy Entire Floor of Brownstone,Brooklyn,Clinton Hill,Brooklyn,40.68514,-73.95976,90.0,279
1,5099,Large Cozy 1 BR Apartment In Midtown East,Manhattan,Murray Hill,Midtown East,40.74767,-73.97500,89.0,75
2,5121,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,Bedford-Stuyvesant,40.68688,-73.95596,90.0,49
3,5178,Large Furnished Room Near B'way,Manhattan,Hell's Kitchen,Manhattan,40.76489,-73.98493,83.0,443
4,5203,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,Upper West Side,40.80178,-73.96723,98.0,118
...,...,...,...,...,...,...,...,...,...
18741,37367714,"Prime downtown location in Little Italy, China...",Manhattan,Chinatown,Manhattan,40.71411,-73.99940,100.0,11
18742,37400191,"Great location, open space (Q line 24/7)",Brooklyn,Sheepshead Bay,Brooklyn,40.59592,-73.95760,78.0,11
18743,37528163,Fantastic home close to everywhere in NYC.,Queens,Flushing,Queens,40.74523,-73.81964,100.0,13
18744,37537425,HarmanBnB,Brooklyn,Bushwick,Brooklyn,40.70517,-73.92237,87.0,12


In [3]:
airbnb_table["review_scores_rating"] = pd.to_numeric(airbnb_table["review_scores_rating"])
filtereddata = airbnb_table[(airbnb_table["review_scores_rating"] >85.0) ] # rating higher than 85
filtereddata = airbnb_table[(airbnb_table["number_of_reviews"] > 10)] # review should be at least 10
filter_airbnb =filtereddata
filter_airbnb

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,name,neighbourhood_group_cleansed,neighbourhood_cleansed,neighbourhood,latitude,longitude,review_scores_rating,number_of_reviews
0,3831,Cozy Entire Floor of Brownstone,Brooklyn,Clinton Hill,Brooklyn,40.68514,-73.95976,90.0,279
1,5099,Large Cozy 1 BR Apartment In Midtown East,Manhattan,Murray Hill,Midtown East,40.74767,-73.97500,89.0,75
2,5121,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,Bedford-Stuyvesant,40.68688,-73.95596,90.0,49
3,5178,Large Furnished Room Near B'way,Manhattan,Hell's Kitchen,Manhattan,40.76489,-73.98493,83.0,443
4,5203,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,Upper West Side,40.80178,-73.96723,98.0,118
...,...,...,...,...,...,...,...,...,...
18741,37367714,"Prime downtown location in Little Italy, China...",Manhattan,Chinatown,Manhattan,40.71411,-73.99940,100.0,11
18742,37400191,"Great location, open space (Q line 24/7)",Brooklyn,Sheepshead Bay,Brooklyn,40.59592,-73.95760,78.0,11
18743,37528163,Fantastic home close to everywhere in NYC.,Queens,Flushing,Queens,40.74523,-73.81964,100.0,13
18744,37537425,HarmanBnB,Brooklyn,Bushwick,Brooklyn,40.70517,-73.92237,87.0,12


In [36]:
filter_airbnb.to_csv('filter_airbnb.csv')

In [69]:
airbnbdata = pd.read_csv('filter_airbnb.csv')
airbnbdata

,Unnamed: 0,id,name,neighbourhood_group_cleansed,neighbourhood_cleansed,neighbourhood,latitude,longitude,review_scores_rating,number_of_reviews
0,0,3831,Cozy Entire Floor of Brownstone,Brooklyn,Clinton Hill,Brooklyn,40.68514,-73.95976,90.0,279
1,1,5099,Large Cozy 1 BR Apartment In Midtown East,Manhattan,Murray Hill,Midtown East,40.74767,-73.97500,89.0,75
2,2,5121,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,Bedford-Stuyvesant,40.68688,-73.95596,90.0,49
3,3,5178,Large Furnished Room Near B'way,Manhattan,Hell's Kitchen,Manhattan,40.76489,-73.98493,83.0,443
4,4,5203,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,Upper West Side,40.80178,-73.96723,98.0,118
...,...,...,...,...,...,...,...,...,...,...
18741,18741,37367714,"Prime downtown location in Little Italy, China...",Manhattan,Chinatown,Manhattan,40.71411,-73.99940,100.0,11
18742,18742,37400191,"Great location, open space (Q line 24/7)",Brooklyn,Sheepshead Bay,Brooklyn,40.59592,-73.95760,78.0,11
18743,18743,37528163,Fantastic home close to everywhere in NYC.,Queens,Flushing,Queens,40.74523,-73.81964,100.0,13
18744,18744,37537425,HarmanBnB,Brooklyn,Bushwick,Brooklyn,40.70517,-73.92237,87.0,12


### We designed our tool to be limited to Manhattan, as it is the destination in NYC where the majority of tourist attractions are, and the most accesible for visitors to travel around the city due to a high volume of subway lines and car services. 

In [72]:
filtereddata = airbnbdata[(airbnbdata["neighbourhood_group_cleansed"] == 'Manhattan') ]
filtereddata
top_rating = filtereddata.sort_values(by=['review_scores_rating','number_of_reviews'], ascending = False)
top_rating_bnb = top_rating[(top_rating['review_scores_rating'] > 85.0)]
top_rating_bnb

,Unnamed: 0,id,name,neighbourhood_group_cleansed,neighbourhood_cleansed,neighbourhood,latitude,longitude,review_scores_rating,number_of_reviews
7536,7536,13246804,One cozy private BR close to the mecca of shop...,Manhattan,Midtown,Manhattan,40.76026,-73.96590,100.0,292
8560,8560,15141938,In a Chelsea loft: A PERFECT private room & bath,Manhattan,Chelsea,Chelsea,40.74348,-73.99365,100.0,219
2399,2399,2123731,Spacious Suite in Midtown West/Times sq.,Manhattan,Hell's Kitchen,Manhattan,40.76466,-73.99325,100.0,198
4206,4206,5353612,Beautiful private suite-like 2FL apt. West Vil...,Manhattan,West Village,West Village,40.73433,-74.00588,100.0,180
10279,10279,18476474,Bright XL room in Hamilton Heights/Sugar Hill,Manhattan,Harlem,Manhattan,40.82444,-73.94525,100.0,145
...,...,...,...,...,...,...,...,...,...,...
15346,15346,27978240,Private/Relaxing Queen PopUp bed 30min2timessqr,Manhattan,Inwood,Inwood,40.86524,-73.92195,86.0,13
15791,15791,28912358,Parisian style 1 bedroom in the Heart of UES!!!,Manhattan,Upper East Side,Manhattan,40.76035,-73.96045,86.0,13
16747,16747,30737182,Central Park Charming Home,Manhattan,Upper West Side,Manhattan,40.77730,-73.97736,86.0,13
9270,9270,16416857,Warm and quiet studio in heart of chinatown,Manhattan,Chinatown,Manhattan,40.71538,-73.99669,86.0,12


### Based on our previous analysis of the safest subway stations in Manhattan, we wanted to help travelers determine what Airbnb locations are accesible to these low-crime transit areas. Based on our list of the top 10 safest subway stations, we determined every Airbnb within 200 meters of each station. With this tool, visitors to NYC will be able to determine the best Airbnb options that are near the safest subway stations in the city!

In [73]:
safest_station = pd.read_csv('safest_final.csv')

In [74]:
safest_station

,station,avg_lat,avg_lon,complaint,subway
0,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0,34 Street - Hudson Yards Station
1,Greenwich St & Morris St,40.70649,-74.01414,8.0,Bowling Green Station
2,2nd Ave & 94th St,40.78294,-73.94798,11.0,96 Street Station - Q
3,Central Park West & 87th St,40.78612,-73.96901,11.0,86 Street Station
4,Central Park West & 81st St,40.78212,-73.97188,15.0,81 Street - Museum of Natural History Station
5,Central Park West & 70th St,40.77494,-73.97714,17.0,72 Street Station
6,Central Park West & 97th St,40.79244,-73.96437,17.0,"96 Street Station - B,C"
7,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0,135 Street Station
8,6th Ave & 16th St,40.73869,-73.99587,23.0,14 Street Station
9,6th Ave & 49th St,40.75929,-73.98085,25.0,47-50 Streets Rockefeller Center Station


# Analyzing the data

In [75]:
from geopy.distance import geodesic
def geodistance(location1, location2):
    distance = geodesic(location1,location2).m
    return distance

In [76]:
import time

count = 0

for id1,row1 in safest_station.iterrows():
    number_of_airbnb = 0
    location1 = (row1['avg_lat'], row1['avg_lon'])
    
    for id2,row2 in top_rating_bnb.iterrows():
        location2 = (row2['latitude'], row2['longitude'])
        if(geodistance(location1,location2) <=200):
            number_of_airbnb = number_of_airbnb + 1
            neighbor = row2['neighbourhood_cleansed']
            
    safest_station.loc[id1,'num_bnb'] = number_of_airbnb
    safest_station.loc[id1,'neighbor'] = neighbor
    #count = count + 1
safest_station

,station,avg_lat,avg_lon,complaint,subway,num_bnb,neighbor
0,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0,34 Street - Hudson Yards Station,2.0,Hell's Kitchen
1,Greenwich St & Morris St,40.70649,-74.01414,8.0,Bowling Green Station,22.0,Financial District
2,2nd Ave & 94th St,40.78294,-73.94798,11.0,96 Street Station - Q,23.0,Upper East Side
3,Central Park West & 87th St,40.78612,-73.96901,11.0,86 Street Station,9.0,Upper West Side
4,Central Park West & 81st St,40.78212,-73.97188,15.0,81 Street - Museum of Natural History Station,7.0,Upper West Side
5,Central Park West & 70th St,40.77494,-73.97714,17.0,72 Street Station,11.0,Upper West Side
6,Central Park West & 97th St,40.79244,-73.96437,17.0,"96 Street Station - B,C",6.0,Upper West Side
7,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0,135 Street Station,28.0,Harlem
8,6th Ave & 16th St,40.73869,-73.99587,23.0,14 Street Station,10.0,Chelsea
9,6th Ave & 49th St,40.75929,-73.98085,25.0,47-50 Streets Rockefeller Center Station,7.0,Theater District


In [77]:
dfsafeairbnbs = pd.DataFrame()
for id1,row1 in safest_station.iterrows():
   
    location1 = (row1['avg_lat'], row1['avg_lon'])
   
    for id2,row2 in top_rating_bnb.iterrows():
        location2 = (row2['latitude'], row2['longitude'])
        if(geodistance(location1,location2) <=200):
            
            dfsafeairbnbs=dfsafeairbnbs.append(row2, ignore_index=True)
#             dfsafeairbnbs.append({'Airbnbname':row2['name']},ignore_index=True)
#             dfsafeairbnbs.append({'latitude':row2['latitude']},ignore_index=True)
#             dfsafeairbnbs.append({'longitude':row2['longitude']},ignore_index=True)
            
            

dfsafeairbnbs

,Unnamed: 0,id,latitude,longitude,name,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,number_of_reviews,review_scores_rating
0,3703.0,4472978.0,40.75531,-73.99945,Charming Studio in a Great location,Manhattan,Hell's Kitchen,Manhattan,141.0,99.0
1,9006.0,16014791.0,40.75536,-73.99950,Clean style apt. Central midtwn near Javits,Midtown,Hell's Kitchen,Manhattan,192.0,90.0
2,17360.0,32240966.0,40.70620,-74.01192,Sonder | Stock Exchange | Sleek 3BR + Lounge,Financial District,Financial District,Manhattan,14.0,100.0
3,17716.0,33072230.0,40.70645,-74.01186,Sonder | Stock Exchange | Sleek 3BR + Laundry,Financial District,Financial District,Manhattan,11.0,100.0
4,11409.0,20518456.0,40.70744,-74.01521,Bright & Luxurious Studio in Financial District,Manhattan,Financial District,Manhattan,43.0,99.0
...,...,...,...,...,...,...,...,...,...,...
120,18024.0,33812619.0,40.75893,-73.98231,Cozy & Classic two-beds room in Times Square,Manhattan,Theater District,Manhattan,19.0,97.0
121,18247.0,34397198.0,40.75845,-73.98026,Tranquility on Times Square VIP Room King Bed,Midtown,Midtown,Manhattan,13.0,97.0
122,13523.0,24075850.0,40.75866,-73.98032,Cozy private room in Times Square/Rockefeller,Manhattan,Midtown,Manhattan,13.0,95.0
123,1918.0,1472002.0,40.76035,-73.98026,SHARED STUDIO AT MANHATTAN'S HEART BY C. PARK,Midtown,Theater District,Manhattan,62.0,92.0


# Visualizing the data

## Once again, below is a visualization of the top 10 safest subway stations (determined by lowest number of complaints within 100m) on the map 

In [78]:
import folium
airbnbmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
airbnbmap

In [79]:
for name, row in safest_station.iterrows():
   
    color = "blue"
    size = 4
    
      # The code below defines a pop-up for each station with details such as 
    # the address, number of bikes, capacity, etc.
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Station Name: </strong>" + row["subway"] + \
           "<br><strong>Number of Complaint: </strong>" + str(row["complaint"] )
        
        
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

  
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                       
                        fill_color = color,
                         popup = popup,
                       ).add_to(airbnbmap)
    
airbnbmap


## And finally, below is a visualization of the best Airbnb options located within 200 meters of the top 10 safest subway stations in Manhattan. The visitors can click on the map to see the popup information about the airbnb locations and choose where they want to live. 

In [80]:
safeairbnbnames = list()
for name, row in dfsafeairbnbs.iterrows():
    color = "red"
    size = 1

    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Airbnb Name: </strong>" + row["name"] + \
           "<br><strong>Rating Scores: </strong>" + str(row["review_scores_rating"] ) + \
           "<br><strong>Number of review: </strong>" + str(row["number_of_reviews"])
       
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

    
    
    # We create a marker on the map and we add it to the map
    folium.Marker(location=[row["latitude"], row["longitude"]], 
                        radius = size,
                        popup = popup, 
                        #fill_opacity = opacity,
                        fill_color = color,
                        icon=folium.Icon(icon='cloud',color='red')
                       ).add_to(airbnbmap)
   
    
airbnbmap


## 4) Conclusion and Further Steps

>### Conclusion

>The conclusion of the study shows that top 10 NYC Subway crime locations, whether highest or lowest, were well spread out throughout Manhattan. Prior to analysis, we believed that the criminal locations would be centered at certain locations. Further we have found safest locations are spread out North to the South on the West side near Central Park of Manhattan and most dangerous locations are spread out from Midtown to Downtown stations.

>To all enthusiastic travelers, we highly recommend staying near Central Park area, where they will have lot of fun activities to do and have safe travel!  
  
 
>### Further Steps

>We believe our program presents a good base for further improvements. It is a good start given that we had to work under a limited system capability, e.g. file space, processing speed, etc. We see the program as having great potential and can be expanded. For example, we can expand the scope of the analysis for all five New York Boroughs, as well as analyzing near 2000 subway entrances, over 4 million complaint data, and 48K Airbnb.  

## 5) Lesson Learned

We recognize the value of large data management, particularly as we were able to apply the lessons learned from the class to produce the program that we know can potentially help visitors of New York City.  By completing this project successfully within the limited parameters provided, e.g. data processing limitation, we created a program that can be add value to NYC visitors.  Data management is a powerful tool for any functional applications and can improve all life application. 

Another lesson learned is that although it was difficult to manage such large data, as we encountered several obstacles to process for the desired output, at the end we were able to develop a functional program that we could not have imagined at the beginning of the semester. We had to go through several iterations of the plans and the codes to come up with our final project outputs. However, through hard work and great teamwork we were able to generate a program and an output that we are proud of.